## Importing all the necessary libraries

In [1]:
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.image import imread
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, save_model
from tensorflow.keras.layers import Input, Dense, Average, BatchNormalization, Flatten
from tensorflow.keras import regularizers
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

## Getting the tomato dataset

In [2]:
dataset_directory_location = 'tomato_dataset'

## Listing the directories present inside tomato dataset

In [3]:
os.listdir(dataset_directory_location)

['test', 'train']

## Getting the training and testing directory location

In [4]:
train_dataset_location = os.path.join(dataset_directory_location, 'train')
test_dataset_location = os.path.join(dataset_directory_location, 'test')

## Listing all the directories present inside 'train' and 'test' directory respectively

In [5]:
os.listdir(train_dataset_location)

['Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___healthy',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']

In [6]:
os.listdir(test_dataset_location)

['Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___healthy',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']

## Viewing a 'tomato image with bacterial spot image'

In [7]:
tomato_bacterial_spot_img = os.path.join(train_dataset_location, 'Tomato___Bacterial_spot')

In [8]:
os.listdir(tomato_bacterial_spot_img)[:1]

['00416648-be6e-4bd4-bc8d-82f43f8a7240___GCREC_Bact.Sp 3110.JPG']

In [ ]:
tomato_bacterial_spot_img_read = imread(os.path.join(tomato_bacterial_spot_img, '00416648-be6e-4bd4-bc8d-82f43f8a7240___GCREC_Bact.Sp 3110.JPG'))

In [ ]:
plt.imshow(tomato_bacterial_spot_img_read)

## Getting the shape of the image

In [ ]:
tomato_bacterial_spot_img_read.shape

## Setting the right image shape of the image for our use case

In [ ]:
correct_image_shape = (224, 224, 3)

In [ ]:
image_gen = ImageDataGenerator(rotation_range=20, 
                               width_shift_range=0.1, 
                               height_shift_range=0.1, 
                               shear_range=0.1, 
                               zoom_range=0.1, 
                               horizontal_flip=True, 
                               fill_mode='nearest',
                               preprocessing_function=preprocess_input)

In [ ]:
train_image_gen = image_gen.flow_from_directory(train_dataset_location, 
                                                target_size=correct_image_shape[:2], 
                                                color_mode='rgb', 
                                                batch_size=16, 
                                                class_mode='categorical')

test_image_gen = image_gen.flow_from_directory(test_dataset_location, 
                                                target_size=correct_image_shape[:2], 
                                                color_mode='rgb', 
                                                batch_size=16, 
                                                class_mode='categorical')

## Creating and training the model

In [ ]:
vgg16_model = load_model('VGG16_model', compile=False)
inceptionV2_model = load_model('inceptionV2_model', compile=False)
cnnfromscratch_model = load_model('cnnfromscratch_model', compile=False)

In [ ]:
vgg16_model._name = 'model1'
inceptionV2_model._name = 'model2'
cnnfromscratch_model._name = 'model3'

In [ ]:
models = [vgg16_model, inceptionV2_model, cnnfromscratch_model]

In [ ]:
model_input = Input(shape=correct_image_shape)

In [ ]:
model_output = [model(model_input) for model in models]

In [ ]:
x = Average()(model_output)

x = BatchNormalization()(x)

x = Dense(128, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = BatchNormalization()(x)

x = Dense(64, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = BatchNormalization()(x)

x = Dense(64, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = BatchNormalization()(x)

x = Dense(32, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(x)
x = BatchNormalization()(x)


output_of_the_model = Dense(10, activation='softmax')(x)

In [ ]:
ensemble_model = Model(model_input, output_of_the_model)

In [ ]:
ensemble_model.summary()

In [ ]:
ensemble_model.compile(loss=CategoricalCrossentropy(), optimizer=Adam(), metrics=['accuracy'])

In [ ]:
ensemble_model.fit(train_image_gen, validation_data=test_image_gen, epochs=10)

In [ ]:
history_df = pd.DataFrame(ensemble_model.history.history)

In [ ]:
history_df[['loss', 'val_loss']].plot()

In [ ]:
history_df[['accuracy', 'val_accuracy']].plot()

## Calculating the accuracy on training and testing data

In [ ]:
ensemble_model.evaluate(train_image_gen)[1]

In [ ]:
ensemble_model.evaluate(test_image_gen)[1]

## Making prediction on sample data

In [ ]:
sample_img_load = image.load_img('./tomato_dataset/test/Tomato___Early_blight/0b494c44-8cd0-4491-bdfd-8a354209c3ae___RS_Erly.B_9561.JPG_0ef2cb9c-711d-46cf-b519-4a047a7bb22a.JPG', target_size=correct_image_shape)

In [ ]:
sample_img_load

In [ ]:
sample_img_to_arr = image.img_to_array(sample_img_load) 

In [ ]:
sample_img_to_arr_preprocess_input = preprocess_input(sample_img_to_arr)

In [ ]:
sample_img_to_arr_expand_dim = np.expand_dims(sample_img_to_arr_preprocess_input, 0) 

In [ ]:
prediction = ensemble_model.predict(sample_img_to_arr_expand_dim)[0]

In [ ]:
pred_result = np.argmax(prediction)

In [ ]:
class_names = ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']

In [ ]:
pred_final_result = class_names[pred_result]

In [ ]:
pred_final_result